In [4]:
import numpy as np
import pdb
import pandas as pd
import pickle
import os
import sys
import csv
import time
import itertools as it
import scipy.io
import h5py
import random

#Assuming same number of samples per user
#Retrieves a chunk of consecutive samples starting from a random point in every class

def custom_split(X, y, test_size = None, randomize = None):
    num_samples = X.shape[0] #rows give the number of samples
    num_observ = X.shape[1]
    classes = np.unique(y)
    classes = classes.astype(int)
    num_classes = classes.size
    aux = np.where(y == classes[0])
    num_samples_user = aux[0].size

    if num_samples != y.size:
        print('X and y sizes are not coherent')
        return [], [], [], []
    
    if test_size == None:
        test_size = 0.2
    
    if randomize == None:
        randomize = True
    
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    indices = []
    
    for i in np.arange(0, num_classes):
        
        test_samples_user = round(num_samples_user*test_size)
        train_samples_user = num_samples_user - test_samples_user
        X_user = X[i*num_samples_user:(i+1)*num_samples_user, :]
        y_user = y[i*num_samples_user:(i+1)*num_samples_user]

        if randomize == True:
            ind = random.sample(range(0, num_samples_user), test_samples_user)    
            ind = sorted(ind)
        else:
            #Create random number to start selected test chunk
            start = random.randint(0, num_samples_user-test_samples_user)            
            end = start + test_samples_user
            ind = np.arange(start, end)
            
        indices = np.append(indices, ind+i*num_samples_user)

    indices = indices.astype(int).tolist()
    mask_train = np.ones(num_samples, np.bool)
    mask_train[indices] = 0
    X_test = X[indices, :]
    X_train = X[mask_train, :]
    y_test = y[indices]
    y_train = y[mask_train]
    
    return np.asarray(X_train), np.asarray(y_train), np.asarray(X_test), np.asarray(y_test), indices